### Prepare Data

In [13]:
import string
from random import shuffle
import numpy as np
x = [[i] for i in range(10)]


FEMALE_FILE = 'Baby_names/female.txt';
MALE_FILE = 'Baby_names/male.txt';
N_ALPHABET = 28; #Add 2 extra for encoding the end of name EON and the dash '-'
MAX_CHARS = 16; #Assume no name is longer than 16 chars

alphabet_id = {}
for i in xrange(26):
    alphabet_id[string.lowercase[i]] = i;

alphabet_id['-'] = 26;
alphabet_id['\n'] = 27;

id2char = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',\
           'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', \
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', '\n'];

def str2onehot(string):
    string = string.lower();
    result = np.zeros((MAX_CHARS, N_ALPHABET));
    
    chars = list(string);
    idx = [alphabet_id[char] for char in chars];
    result[range(len(string)), idx] = 1;
    return result;
input_male = []
target_male = []
sequence_len = []
with open(MALE_FILE) as fp:
    for line in fp:
        if (line[0]!="#" and line[0]!="\n"):
            input_male.append(str2onehot(line[:-1]))
            target_male.append(str2onehot(line[1:]))
            sequence_len.append(len(line)-1);
n = len(input_male);
idx = [i for i in range(n)]
shuffle(idx)
input_male = [input_male[i] for i in idx]
target_male = [target_male[i] for i in idx]
sequence_len = [sequence_len[i] for i in idx]

### Build the LSTM graph

In [14]:
import tensorflow as tf
tf.reset_default_graph()

max_chars = max([name.shape[0] for name in input_male]);
#Placeholder for input and target, [Batch Size, Sequence Length, Input Dimension]
data = tf.placeholder(tf.float32, [None, max_chars, N_ALPHABET])
target = tf.placeholder(tf.float32, [None, max_chars, N_ALPHABET])

num_hidden = 50
time_steps = max_chars

lstmCell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)
#prediction one char
x = tf.placeholder(tf.float32, [1, N_ALPHABET])
m = lstmCell.zero_state(1, dtype=tf.float32)
out_char, state = lstmCell(x, m);


seqlen = tf.placeholder(tf.int32, [None])

#unroll the LSTM network
outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=data, sequence_length=seqlen, dtype=tf.float32)

# Define weights
out_size = N_ALPHABET;

# Weights for each timestep output, using the same weight
weight = tf.Variable(tf.truncated_normal([num_hidden, out_size], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[out_size]))
char_prediction = tf.nn.softmax(tf.matmul(out_char, weight) + bias)

outputs = tf.reshape(outputs, [-1, num_hidden])
prediction = tf.nn.softmax(tf.matmul(outputs, weight) + bias)
prediction = tf.reshape(prediction, [-1, time_steps, out_size])

cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
#cross_entropy = tf.reduce_mean(cross_entropy)

#cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
final_prediction = tf.argmax(prediction, 2)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
minimize = optimizer.minimize(cross_entropy)

mistakes = tf.not_equal(tf.argmax(target, 2), tf.argmax(prediction, 2))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

### Execute the graph

In [15]:
from numpy import random
def pred2name(pred):
    #ndarray of n_time_steps * N_ALPHABET
    name = [id2char[char_id] for char_id in pred];
    return "".join(name)

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

batch_size = 100
no_of_batches = int(len(input_male)/batch_size)
epoch = 2000

for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, seql = input_male[ptr:ptr+batch_size], target_male[ptr:ptr+batch_size], sequence_len[ptr:ptr+batch_size]   
        ptr+=batch_size
        result = sess.run([minimize, cross_entropy, final_prediction, error], {data: inp, target: out, seqlen: seql})
        #print(result[3])
        
    print(result[1])
    prediction = result[2];
    idx = random.choice(range(prediction.shape[0]), size=2);
    preds = prediction[idx]
    if (i % 10) == 0:
        for pred in preds:
            sample_name = pred2name(pred)
            print repr(sample_name)
            
        print "Epoch - ",str(i)

#sess.close();



1896.68
'ee\n\n\n\nrrrrrrrrrr'
'\ne\n\nrrrrrrrrrrrr'
Epoch -  0
1646.05
1618.54
1599.98
1583.82
1569.88
1556.01
1541.07
1522.44
1498.78
1470.33
'aeer\n\n\naaaaaaaaa'
'arrr\n\naaaaaaaaaa'
Epoch -  10
1439.25
1411.21
1387.78
1368.02
1351.22
1336.89
1324.38
1313.26
1303.32
1294.31
'anen\naaaaaaaaaaa'
'reen\naaaaaaaaaaa'
Epoch -  20
1286.01
1278.27
1270.97
1264.01
1257.34
1250.92
1244.72
1238.75
1233.0
1227.45
'aitton\naaaaaaaaa'
'areaaaaaaaaaaaaa'
Epoch -  30
1222.11
1216.96
1211.98
1207.17
1202.51
1197.98
1193.57
1189.26
1185.05
1180.92
'anar\n\n\naaaaaaaaa'
'aree\ne\ne\n\naaaaaa'
Epoch -  40
1176.85
1172.83
1168.86
1164.94
1161.05
1157.22
1153.43
1149.69
1145.99
1142.34
'ansin\n\n\naaaaaaaa'
'halan\naaaaaaaaaa'
Epoch -  50
1138.73
1135.16
1131.62
1128.1
1124.61
1121.15
1117.71
1114.29
1110.89
1107.52
'ardin\naaaaaaaaaa'
'erein\n\n\naaaaaaaa'
Epoch -  60
1104.19
1100.89
1097.63
1094.43
1091.27
1088.17
1085.12
1082.13
1079.19
1076.3
'harne\nuuuuuuuuuu'
'arel\nuuuuuuuuuuu'
Epoch -  70
1073

## Generate Baby Name

In [45]:
batch_size = 1;
input_size = N_ALPHABET;

n = 50;
def char2onehot(char):
    char = char.lower();
    result = np.zeros((1, N_ALPHABET));
    idx = alphabet_id[char];
    result[0, idx] = 1;
    return result;

def samplePred(pred):
    dist = pred[0, ]
    idx_min = dist.argsort()[:(len(dist)/2)]
    pred[0, idx_min] = 0
    pred[0, ] = pred[0, ]/sum(pred[0, ])
    char_id = np.random.choice(N_ALPHABET, 1, p=pred[0, ])
    return char_id
    
for i in range (n):
    init_char = "a"
    last_char = init_char[0]
    init_char = init_char[1:]
    count = 0
    name = last_char;
    last_input = char2onehot(last_char);
    current_state = (np.zeros((1, num_hidden)), np.zeros((1, num_hidden)));
    while (last_char != '\n') and (count < 17):
        count = count+1;
        res = sess.run([char_prediction, state], {x:last_input, m:current_state} );
        char_id = samplePred(res[0])
        if len(init_char) > 0:
            last_char = init_char[0]
            init_char = init_char[1:]
        else:
            last_char = id2char[char_id];
        name += last_char;
        last_input = char2onehot(last_char);
        current_state = res[1];
    print name;


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:36: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


ahzrpgewhzggzisayi
a

ahhzfshhhbbztpapne
ayupipdlvvipgvkezk
aokzznhgeozisvbzdz
avliglvuizzipdtvgh
anuleudviiyitk-pzl
aluprpvinhgziivigi
azuindil-invzzshhz
an

avdvdhepoipnarh

ailvzzzzittshzpgvi
azzdhrfevvbgozhszh
aevzzldsrh

avddzfvnggevgeewri
avn

audvvniinpi

aupdrivyznsezhheev
adhoubwjao

ardgviiz

avznidvnipglgzegge
arillnuinp

avidzsnecilizzskls
aiivilivviiippnghh
ar

adinljsnihkzpzzitl
apdzrlo

apevvzhzhhzgzzisze
aizdpppievhvwvzgrg
azlhdfghpmpazelvjn
azzizgluzzmip

apldpnlhlvzwosscpa
ap

aedvewvizwplr

ahrpvowvozzvsgkeni
ai

avridvjiiidipvhziv
aoudneiiilnuliitvi
ahizjzlisjlhscknvj
auzdozplizdiviwnnh
az

a

azzilvnshdzhhphezz
avpiwzviyzzgtrsrhs
auddlovwiwilvzhtk-
auevridnhppvi-l

aogzlslurhvblovz

arill

azzshwzvizgevtiiji
aumilm



In [68]:
sess.close();

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.]])

In [44]:
from random import shuffle
idx = [i for i in range(10)]
shuffle(idx)


In [4]:
%autosave 10000

Autosaving every 10000 seconds


In [43]:
type(x)

NoneType

In [15]:
sess.close()

In [99]:
a = np.array([9, 4, 4, 3, 3, 9, 0, 4, 6, 0])
ind = np.argpartition(a, -5)[-5:]
ind
a[ind]

array([4, 4, 9, 6, 9])